## Definizione path ProstateMRI

In [ ]:
pathProstateMRI = "../ProstateMRI_PerMMSEG/"

## SALVATAGGIO TEST SET NO CROP

In [ ]:
import os
import shutil
from tqdm import tqdm

# Percorso principale del dataset
pathDataset = "../DATASET/"

# Leggi il file con i nomi dei pazienti del test set
with open("./pazientiTumTest.txt", "r") as file:
    # Crea una lista con i nomi dei pazienti, rimuovendo spazi vuoti
    ListaPazientiTest = [line.strip() for line in file.readlines()]

# Crea la directory di destinazione se non esiste
os.makedirs(pathProstateMRI + "testSetNoCrop/", exist_ok=True)

# Tipi di dati da copiare per ogni paziente
data_types = ["stacked", "adc", "hbv", "t2w", "manual"]

# Copia le directory per ciascun tipo di dato per ogni paziente
for paziente in tqdm(ListaPazientiTest):
    for data_type in data_types:
        source_path = os.path.join(pathDataset, paziente, data_type)
        target_path = os.path.join(pathProstateMRI, "testSetNoCrop", paziente, data_type)
        shutil.copytree(source_path, target_path)



In [ ]:
import os
import shutil
from tqdm import tqdm

# Definizione dei percorsi principali
pathDataset = "../DATASET/"  # Percorso del dataset sorgente

# Lettura dei nomi dei pazienti dal file txt
with open("./pazientiSaniTest.txt", "r") as fileTest:
    ListaPazientiTest = [line.strip() for line in fileTest.readlines()]  # Rimozione degli spazi bianchi

# Creazione della directory di destinazione se non esiste
os.makedirs(os.path.join(pathProstateMRI, "testSetNoCrop/"), exist_ok=True)

# Tipi di dati da copiare per ogni paziente
data_types = ["stacked", "adc", "hbv", "t2w", "manual"]

# Copia delle cartelle per ogni paziente nel test set
for paziente in tqdm(ListaPazientiTest, desc="Copying patient data"):
    for data_type in data_types:
        source_path = os.path.join(pathDataset, paziente, data_type)  # Percorso sorgente dei dati
        target_path = os.path.join(pathProstateMRI, "testSetNoCrop", paziente, data_type)  # Percorso di destinazione
        shutil.copytree(source_path, target_path)  # Copia della cartella


## Salvataggio Tumorali

In [ ]:
import os
import shutil
from tqdm import tqdm
import numpy as np
from PIL import Image

# Definizione dei percorsi principali
pathDataset = "../DATASET/"

# Definisci le directory
dirs = ['adc_dir', 'hbv_dir', 't2w_dir', 'stacked_dir', 'ann_dir']
sub_dirs = ['train', 'val', 'test']

# Crea le directory
for dir in dirs:
    for sub_dir in sub_dirs:
        os.makedirs(os.path.join(pathProstateMRI, dir, sub_dir), exist_ok=True)

# Leggi i file dei pazienti
patient_files = ['./pazientiTumTrain.txt', './pazientiTumVal.txt', './pazientiTumTest.txt']
patient_lists = [open(file, 'r').read().splitlines() for file in patient_files]

# Copia i file nelle directory corrispondenti
for patients, sub_dir in zip(patient_lists, sub_dirs):
    for patient in tqdm(patients):
        patient = str(patient)
        tumor_slices = []
        slices = sorted(os.listdir(os.path.join(pathDataset, patient, 'manual')))

        # Identifica le slice tumorali
        for slice in slices:
            mask_path = os.path.join(pathDataset, patient, 'manual', slice)
            if os.path.exists(mask_path):
                mask = np.array(Image.open(mask_path))
                if mask.sum() > 0:
                    tumor_slices.append(slice)

        # Aggiungi la slice prima e dopo la sequenza di slice tumorali
        first_tumor_slice_index = slices.index(tumor_slices[0])
        last_tumor_slice_index = slices.index(tumor_slices[-1])
        if first_tumor_slice_index > 0:
            tumor_slices.append(slices[first_tumor_slice_index - 1])
        if last_tumor_slice_index < len(slices) - 1:
            tumor_slices.append(slices[last_tumor_slice_index + 1])

        # Evita duplicati
        tumor_slices = list(set(tumor_slices))

        for dir in dirs:
            # Copia le maschere da 'manual' a 'ann_dir'
            if dir == 'ann_dir':
                for slice in tumor_slices:
                    shutil.copy(os.path.join(pathDataset, patient, 'manual', slice), os.path.join(pathProstateMRI, dir, sub_dir, patient + "_" + slice))
            else:
                for slice in tumor_slices:
                    shutil.copy(os.path.join(pathDataset, patient, dir.split('_')[0], slice), os.path.join(pathProstateMRI, dir, sub_dir, patient + "_" + slice))


## Salvataggio Sani

In [ ]:
import os
import shutil
from tqdm import tqdm

# Percorso del dataset
pathDataset = "../DATASET/"

# Definisci le directory
dirs = ['adc_dir', 'hbv_dir', 't2w_dir', 'stacked_dir', 'ann_dir']
sub_dirs = ['train', 'val', 'test']

# Crea le directory
for dir in dirs:
    for sub_dir in sub_dirs:
        os.makedirs(os.path.join(pathProstateMRI, dir, sub_dir), exist_ok=True)

# Leggi i file dei pazienti
patient_files = ['./pazientiSaniTrain.txt', './pazientiSaniVal.txt', './pazientiSaniTest.txt']
patient_lists = [open(file, 'r').read().splitlines() for file in patient_files]

# Copia i file nelle directory corrispondenti
for patients, sub_dir in zip(patient_lists, sub_dirs):
    for patient in tqdm(patients):
        patient = str(patient)
        slice_to_copy = ["slice_" + str(i) + ".png" for i in range(int(len(os.listdir(os.path.join(pathDataset, patient, 'adc')))/4), int(len(os.listdir(os.path.join(pathDataset, patient, 'adc')))*3/4), 2)]
        for dir in dirs:
            # Copia le maschere da 'manual' a 'ann_dir'
            if dir == 'ann_dir':
                for slice in slice_to_copy:
                    shutil.copy(os.path.join(pathDataset, patient, 'manual', slice), os.path.join(pathProstateMRI, dir, sub_dir, patient + "_" + slice))
            else:
                for slice in slice_to_copy:
                    shutil.copy(os.path.join(pathDataset, patient, dir.split('_')[0], slice), os.path.join(pathProstateMRI, dir, sub_dir, patient + "_" + slice))


## Crop e Resize

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

def findBorders(t2w):
    # FIND BLACK BORDERS IN T2W CHANNEL
    # Make t2w boolean
    t2w_bool = t2w > 0
    # Find non-black rows and columns
    rows = np.any(t2w_bool, axis=1)
    cols = np.any(t2w_bool, axis=0)
    # Get first and last non-black row and column indexes
    rmin, rmax = np.where(rows)[0][[0, -1]] if np.any(rows) else (0, t2w.shape[0])
    cmin, cmax = np.where(cols)[0][[0, -1]] if np.any(cols) else (0, t2w.shape[1])
    return rmin, rmax, cmin, cmax

def cropResizeImages(adc, hbv, t2w, mask):
    # Trova i bordi neri nel canale t2w
    rmin_t2w, rmax_t2w, cmin_t2w, cmax_t2w = findBorders(t2w)
    
    # Calcola la dimensione del crop
    cropDimension = min(rmax_t2w - rmin_t2w, cmax_t2w - cmin_t2w)
    if cropDimension % 2 != 0:  # Assicurati che cropDimension sia pari
        cropDimension -= 1

    # Ottieni il centro della regione non nera
    center_x = (cmin_t2w + cmax_t2w) // 2
    center_y = (rmin_t2w + rmax_t2w) // 2

    # Ottieni le coordinate di inizio e fine per il cropping
    start_x = max(center_x - cropDimension // 2, 0)
    start_y = max(center_y - cropDimension // 2, 0)
    end_x = min(start_x + cropDimension, t2w.shape[1])
    end_y = min(start_y + cropDimension, t2w.shape[0])

    # Esegui il crop quadrato su tutti i canali e sulla maschera
    adc_cropped = adc[start_y:end_y, start_x:end_x]
    hbv_cropped = hbv[start_y:end_y, start_x:end_x]
    t2w_cropped = t2w[start_y:end_y, start_x:end_x]
    mask_cropped = mask[start_y:end_y, start_x:end_x]

    # Ridimensiona le immagini croppate a 256x256
    resizeShape = (256, 256)
    adc_resized = np.array(Image.fromarray(adc_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    hbv_resized = np.array(Image.fromarray(hbv_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    t2w_resized = np.array(Image.fromarray(t2w_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    mask_resized = np.array(Image.fromarray(mask_cropped, mode='L').resize(resizeShape, Image.NEAREST), dtype=np.uint8)

    return adc_resized, hbv_resized, t2w_resized, mask_resized



In [ ]:
# Elenco dei set di dati
setTutti = ["train", "val", "test"]

# Processa ogni set di dati
for actSet in tqdm(setTutti):
    # Ottieni l'elenco delle slice
    slices = os.listdir(os.path.join(pathProstateMRI, "stacked_dir", actSet))
    for slice in slices:
        # Carica l'immagine e la maschera
        i = np.array(Image.open(os.path.join(pathProstateMRI, "stacked_dir", actSet, slice)))
        m = np.array(Image.open(os.path.join(pathProstateMRI, "ann_dir", actSet, slice)))
        
        # Separa i canali dell'immagine
        adc, hbv, t2w = i[:,:,0], i[:,:,1], i[:,:,2]
        
        # Esegui il crop e il resize
        adc_resized, hbv_resized, t2w_resized, m_resized = cropResizeImages(adc, hbv, t2w, m)

        # Creazione della stacked
        i_resized = np.stack([adc_resized, hbv_resized, t2w_resized], axis=-1)
        
        # Salva le immagini ridimensionate
        Image.fromarray(adc_resized).save(os.path.join(pathProstateMRI, "adc_dir", actSet, slice))
        Image.fromarray(hbv_resized).save(os.path.join(pathProstateMRI, "hbv_dir", actSet, slice))
        Image.fromarray(t2w_resized).save(os.path.join(pathProstateMRI, "t2w_dir", actSet, slice))
        Image.fromarray(i_resized).save(os.path.join(pathProstateMRI, "stacked_dir", actSet, slice))
        Image.fromarray(m_resized).save(os.path.join(pathProstateMRI, "ann_dir", actSet, slice))

## Rearrange Train/Val per slice (non per pazienti)

In [ ]:
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
import shutil

# Elenco delle slice nei set di training e validation
slice_train = os.listdir(os.path.join(pathProstateMRI, "stacked_dir", "train"))
slice_val = os.listdir(os.path.join(pathProstateMRI, "stacked_dir", "val"))

# Classificazione delle slice come tumorali o sane
slice_tumorali = []
slice_sane = []

def classifica_slice(lista_slice, tipo_set):
    """ Classifica le slice in tumorali o sane in base alla maschera associata. """
    for nome_slice in lista_slice:
        percorso_maschera = os.path.join(pathProstateMRI, "ann_dir", tipo_set, nome_slice)
        maschera = np.array(Image.open(percorso_maschera))
        if np.sum(maschera) > 0:
            slice_tumorali.append(percorso_maschera)
        else:
            slice_sane.append(percorso_maschera)

# Classifica le slice nei set di training e validation
classifica_slice(slice_train, "train")
classifica_slice(slice_val, "val")

# Creazione delle nuove directory per i dati organizzati
os.makedirs(os.path.join(pathProstateMRI, "ann_dir_nuovo", "train"), exist_ok=True)
os.makedirs(os.path.join(pathProstateMRI, "ann_dir_nuovo", "val"), exist_ok=True)
os.makedirs(os.path.join(pathProstateMRI, "stacked_dir_nuovo", "train"), exist_ok=True)
os.makedirs(os.path.join(pathProstateMRI, "stacked_dir_nuovo", "val"), exist_ok=True)

# Divisione delle slice tumorali e sane in 80% train e 20% val separatamente
train_tumorali = []
val_tumorali = []
train_sane = []
val_sane = []

# Mischia casualmente le liste
np.random.shuffle(slice_tumorali)
np.random.shuffle(slice_sane)

# Divisione in train e val
train_tumorali = slice_tumorali[:int(len(slice_tumorali)*0.8)]
val_tumorali = slice_tumorali[int(len(slice_tumorali)*0.8):]
train_sane = slice_sane[:int(len(slice_sane)*0.8)]
val_sane = slice_sane[int(len(slice_sane)*0.8):]

# Copia le slice nei nuovi directory organizzati
for slice in train_tumorali + train_sane:
    nome_slice = os.path.basename(slice)
    nome_set_sorgente = slice.split("\\")[-2]  # Modifica se necessario per il tuo sistema
    shutil.copy(slice, os.path.join(pathProstateMRI, "ann_dir_nuovo", "train", nome_slice))
    shutil.copy(os.path.join(pathProstateMRI, "stacked_dir", nome_set_sorgente, nome_slice),
                os.path.join(pathProstateMRI, "stacked_dir_nuovo", "train", nome_slice))

for slice in val_tumorali + val_sane:
    nome_slice = os.path.basename(slice)
    nome_set_sorgente = slice.split("\\")[-2]  # Modifica se necessario per il tuo sistema
    shutil.copy(slice, os.path.join(pathProstateMRI, "ann_dir_nuovo", "val", nome_slice))
    shutil.copy(os.path.join(pathProstateMRI, "stacked_dir", nome_set_sorgente, nome_slice),
                os.path.join(pathProstateMRI, "stacked_dir_nuovo", "val", nome_slice))

# Copia la cartella del test set così com'è da stacked_dir a stacked_dir_nuovo e da ann_dir a ann_dir_nuovo
shutil.copytree(os.path.join(pathProstateMRI, "ann_dir", "test"), os.path.join(pathProstateMRI, "ann_dir_nuovo", "test"))
shutil.copytree(os.path.join(pathProstateMRI, "stacked_dir", "test"), os.path.join(pathProstateMRI, "stacked_dir_nuovo", "test"))


In [ ]:
# rinomina stacked_dir in stacked_dir_vecchio e ann_dir in ann_dir_vecchio e stacked_dir_nuovo in stacked_dir e ann_dir_nuovo in ann_dir
os.rename(os.path.join(pathProstateMRI, "stacked_dir"), os.path.join(pathProstateMRI, "stacked_dir_vecchio"))
os.rename(os.path.join(pathProstateMRI, "ann_dir"), os.path.join(pathProstateMRI, "ann_dir_vecchio"))

os.rename(os.path.join(pathProstateMRI, "stacked_dir_nuovo"), os.path.join(pathProstateMRI, "stacked_dir")) 
os.rename(os.path.join(pathProstateMRI, "ann_dir_nuovo"), os.path.join(pathProstateMRI, "ann_dir"))

In [ ]:
# cancella stacked_dir_vecchio e ann_dir_vecchio
shutil.rmtree(os.path.join(pathProstateMRI, "stacked_dir_vecchio"))
shutil.rmtree(os.path.join(pathProstateMRI, "ann_dir_vecchio"))